In [1]:
import pandas as pd
from datetime import datetime

## Submissions

In [2]:
subs = pd.read_csv('../data/morocco/submissions.csv', low_memory=False)

# drop unnecessary columns
subs.drop(columns=['Unnamed: 0', 'media', 'media_embed', 'thumbnail'], inplace=True)

# delete rows with [deleted by user] or [ Removed by Reddit ] in title
subs = subs[subs['title'] != '[deleted by user]']
subs = subs[subs['title'] != '[ Removed by Reddit ]']

# replace [deleted] and [removed] with Empty str in selftext and author
subs['selftext'] = subs['selftext'].replace(['[deleted]', '[removed]', 'Title'], None)
subs['author'] = subs['author'].replace('[deleted]', None)


# drop duplicates
subs.drop_duplicates(subset=['id'], inplace=True)
subs.reset_index(drop=True, inplace=True)

# change created_utc to datetime type
subs['created_utc'] = pd.to_datetime(subs['created_utc'])


In [3]:
#subs.to_csv('../data/cleaned/submissions.csv', index=False)

## Comments

In [4]:
comments = pd.read_csv('../data/morocco/comments.csv', low_memory=False)

# drop unnecessary columns
comments.drop(columns=['Unnamed: 0', 'subreddit_id', 'subreddit'], inplace=True)

# replace [deleted] and [removed] with Empty str in author
comments['author'] = comments['author'].replace('[deleted]', None)

# remove AutoModerator comments
comments = comments[comments['author'] != 'AutoModerator']

# drop rows with [deleted] or [removed] in body
comments = comments[comments['body'] != '[deleted]']
comments = comments[comments['body'] != '[removed]']

# drop duplicates
comments.drop_duplicates(subset=['id'], inplace=True)
comments.reset_index(drop=True, inplace=True)

# change created_utc to datetime type
comments['created_utc'] = pd.to_datetime(comments['created_utc'])

# format link_id and parent_id to match submission id
comments['link_id'] = comments['link_id'].str.split('_').str[1]
comments['parent_id'] = comments['parent_id'].str.split('_').str[1]


In [5]:
len(comments)

870662

In [6]:
comments.to_csv('../data/cleaned/comments.csv', index=False)